In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

1. 收益拆分

In [ ]:
xlsx_file = '/Users/willazhang/Desktop/codes/willa_env/um vs bm/Big FIFO PnL 报告.xlsx' 
raw_df = pd.read_excel(xlsx_file,sheet_name=2)

### 不同交易策略的投资组合 
# port_name = 'AM-AR-Binance-CCL-30'
# port_name = 'AM-AR-BinanceBTC-ZenX-12'
# port_name = 'AM-MM-Binance-Daphne-02'
# port_name = 'AM-HFT-Binance-1617-02'
port_name = 'AM-CTA-BinanceBTC-binquant-22'



port = raw_df[raw_df['投资组合'] == port_name]
port = port[['投资组合','账户类型','资产类别','标的币种','结算币种','盈亏类别','期间总盈亏($)','期间总盈亏(币)']] 

port['盈亏类别'] = port['盈亏类别'].replace('期货','交易持仓')
port['盈亏类别'] = port['盈亏类别'].replace('现货','交易持仓')


# 注意币本位交易投组 要剔除BTC 
if port_name.split('-')[2] == 'BinanceBTC':
    trading_port = port[(port['标的币种']!='BTC')&(port['盈亏类别']=='交易持仓')] 
    port = port[(port['盈亏类别']!='交易持仓')]
    port = pd.concat([port,trading_port],ignore_index=True) 


# u本位账户，不用剔除BTC
port1 = port.copy()
p1 = port1.groupby(['盈亏类别'])['期间总盈亏($)'].sum().reset_index()

total_pnl = np.abs(p1['期间总盈亏($)'].sum())
p1['盈亏占比(%)'] = p1['期间总盈亏($)']/total_pnl *100


p1 = p1.sort_values('期间总盈亏($)', ascending=False)
p1 = p1.reset_index(drop=True)

# 加上最后一行总计
p1.loc[len(p1)] = {
    '盈亏类别': '总计',
    '期间总盈亏($)': p1['期间总盈亏($)'].sum(),
    '盈亏占比(%)': p1['盈亏占比(%)'].sum()
}

p1.to_csv('p5_trading.csv', index=False)
print('\n', p1)

In [ ]:
# 1. 准备数据（根据图片中的表格）
df = p1.iloc[:-1]  # 去掉总计行
df = df.set_index('盈亏类别')
df = df.rename(index={ 
    # "盈亏类别"
    "交易持仓":"Trading P&L",
    "资金费": "Funding Fee", 
    "交易手续费": "Transaction Fee",
    "利息":"interest",
    "质押收益":"staking income",
    "奖励金":"Bonus",
    })
print('\n',df)

df = df.sort_values(by = '期间总盈亏($)', ascending=False)
# 2. 创建画布（深色背景风格，与图片一致）
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(10, 6))

# 3. 绘制柱状图（优化正负值显示）
bars = ax.bar(
    x=df.index,
    height=df["期间总盈亏($)"],
    color=['#1f77b4' if val >=0 else '#d62728' for val in df["期间总盈亏($)"]], # 正蓝负红
    edgecolor='white')

# 4. 添加数值标签
for bar in bars: 
    height = bar.get_height() 
    va = 'bottom' if height >=0 else 'top' 
    # 正值标签在柱顶，负值在柱底 
    offset = 30 if height >=0 else -30 
    # 避免标签与轴线重叠 
    ax.text( bar.get_x() + bar.get_width()/2, 
            height + offset, f"{height:,.2f}", 
            ha='center', 
            va=va, 
            color='green', 
            fontsize=10 )
    
# 5. 坐标轴优化
ax.axhline(0, color='white', linestyle='--', linewidth=0.5) 
# 零线
ax.set_ylabel('period P&L($)', labelpad=10)
ax.set_title('P&L Category', pad=20)
ax.tick_params(axis='x', rotation=15) # X轴标签旋转防重叠

# 6. 网格和边框
ax.grid(axis='y', linestyle=':', alpha=0.5)
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)
    
plt.tight_layout()
plt.show()

___

In [ ]:
2. 每个组别内的币种分析, 含交易持仓(期货、现货)

In [ ]:
# ticker范围: {'资金费','交易手续费','利息','交易持仓','奖励金'}
ticker = '交易手续费'
specific_df = port[port['盈亏类别']== ticker]

p2 = specific_df.groupby(['资产类别','标的币种','结算币种','盈亏类别'])['期间总盈亏($)'].sum().reset_index()

# 币本位交易投组 要剔除BTC计算交易持仓收益，计算其他项目(利息、资金费等)不用剔除BTC
if (ticker == '交易持仓')&(port_name.split('-')[2] == 'BinanceBTC'):
    p2 = p2[(p2['标的币种']!='BTC')]  

# # 定义资产名称列
df = p2.copy()
df['资产名称'] = df.apply( 
    lambda row: ( 
        f"{row['标的币种']}-Perp"  
        if (row['资产类别'] == '期货') and (row['标的币种'] == row['结算币种']) 
        else ( 
            f"{row['标的币种']}{row['结算币种']}-Perp" 
            if (row['资产类别'] == '期货') and (row['标的币种'] != row['结算币种'])
            else row['标的币种'] # 现货情况 
        ) 
    ), 
    axis=1
)

print(df.sort_values('标的币种',ascending=False).reset_index(drop=True))


# 币种归类
# 替换质押币
df["币种类别"] = df["标的币种"]
df["币种类别"] = df["币种类别"].replace("WBETH", "ETH")
df["币种类别"] = df["币种类别"].replace("BETH", "ETH")
df["币种类别"] = df["币种类别"].replace("BNSOL", "SOL")
df["币种类别"] = df["币种类别"].replace("OKSOL", "SOL")
df["币种类别"] = df["币种类别"].replace("FDUSD", "USDT")
df["币种类别"] = df["币种类别"].replace("USDC", "USDT")


# 聚合 ETH 相关的盈亏
grouped_df = df.groupby(["币种类别", "盈亏类别"]).agg({
    "期间总盈亏($)": "sum", # 累加盈亏
    }).reset_index()

print("\n===== 聚合后数据 =====")
grouped_df = grouped_df.sort_values(by=['期间总盈亏($)'], ascending=False).reset_index(drop=True)
print('\n',grouped_df,'\n',f"有{len(grouped_df['币种类别'].unique())}个币种类别，分别是: ",grouped_df['币种类别'].unique())

funding_pnl = np.abs(grouped_df['期间总盈亏($)'].sum())
grouped_df['盈亏占比(%)'] = grouped_df['期间总盈亏($)']/funding_pnl *100
df = grouped_df.sort_values('期间总盈亏($)',ascending=False)

# 过滤掉占比小于0.3%的币种
df = df[np.abs(df['盈亏占比(%)'])>0.3].reset_index(drop=True) 
new_funding_pnl = np.abs(df['期间总盈亏($)'].sum())
df['盈亏占比(%)'] = df['期间总盈亏($)']/new_funding_pnl *100
df = df.sort_values('期间总盈亏($)',ascending=False)
df = df.reset_index(drop=True)


df.loc[len(df)] = {
    '币种类别':'总计',
    '盈亏类别': f'{ticker}',
    '期间总盈亏($)': df['期间总盈亏($)'].sum(),
    '盈亏占比(%)': df['盈亏占比(%)'].sum()
}
print('\n',df)
df.to_csv(f'p5_{ticker}.csv', index=False)


In [ ]:
# 绘图
# 1. 准备数据
df = df.iloc[:-1]  # 去掉总计行
df = df.set_index('币种类别')
df = df[['期间总盈亏($)','盈亏占比(%)']].sort_values(by = '期间总盈亏($)', ascending=False)
print(df)

# 2. 创建画布（深色背景风格，与图片一致）
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(56, 6))

# 3. 绘制柱状图（优化正负值显示）
bars = ax.bar(
    x=df.index,
    height=df["期间总盈亏($)"],
    color=['#1f77b4' if val >=0 else '#d62728' for val in df["期间总盈亏($)"]], # 正蓝负红
    edgecolor='white')

# 4. 添加数值标签
for bar in bars: 
    height = bar.get_height() 
    va = 'bottom' if height >=0 else 'top' 
    # 正值标签在柱顶，负值在柱底 
    offset = 0.1 if height >=0 else -0.1 
    # 避免标签与轴线重叠 
    ax.text( bar.get_x() + bar.get_width()/2, 
            height + offset, f"{height:,.2f}", 
            ha='center', 
            va=va, 
            color='green', 
            fontsize=10 )
    
# 5. 坐标轴优化
ax.axhline(0, color='white', linestyle='--', linewidth=0.5) 
# 零线
ax.set_ylabel('period P&L($)', labelpad=10)
ax.set_title('P&L Category', pad=20)
ax.tick_params(axis='x', rotation=15) # X轴标签旋转防重叠

# 6. 网格和边框
ax.grid(axis='y', linestyle=':', alpha=0.5)
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)
    
plt.tight_layout()
plt.show()

___

3. 截面投资组合收益相似性分析： 含有交易持仓(期货现货)

ticker范围: {'资金费','交易手续费','利息','奖励金','交易持仓'}

3.1 投组总体收益来源相关性分析

In [ ]:
# 5大收益来源的组合相似性比较
df1 = pd.read_csv('p1_trading.csv')[['盈亏类别','期间总盈亏($)']].iloc[:-1] # 去掉最后一行总计和最后一列盈亏占比
df2 = pd.read_csv('p2_trading.csv')[['盈亏类别','期间总盈亏($)']].iloc[:-1]
df3 = pd.read_csv('p3_trading.csv')[['盈亏类别','期间总盈亏($)']].iloc[:-1]
df4 = pd.read_csv('p4_trading.csv')[['盈亏类别','期间总盈亏($)']].iloc[:-1]
df5 = pd.read_csv('p5_trading.csv')[['盈亏类别','期间总盈亏($)']].iloc[:-1]


# 合并三个投资组合
combined_df = df1.merge(df2, on='盈亏类别', how='outer', suffixes=('_投组1', '_投组2'))
df3 = df3.rename(columns={ '期间总盈亏($)':'期间总盈亏($)_投组3','盈亏占比(%)': '盈亏占比(%)_投组3'})
df4 = df4.rename(columns={ '期间总盈亏($)':'期间总盈亏($)_投组4','盈亏占比(%)': '盈亏占比(%)_投组4'})
df5 = df5.rename(columns={ '期间总盈亏($)':'期间总盈亏($)_投组5','盈亏占比(%)': '盈亏占比(%)_投组5'})

combined_df = combined_df.merge(df3, on='盈亏类别', how='outer')
combined_df = combined_df.merge(df4, on='盈亏类别', how='outer')
combined_df = combined_df.merge(df5, on='盈亏类别', how='outer')


combined_df = combined_df.fillna(0)
combined_df.set_index('盈亏类别', inplace=True)
combined_df = combined_df[['期间总盈亏($)_投组1','期间总盈亏($)_投组2','期间总盈亏($)_投组3','期间总盈亏($)_投组4','期间总盈亏($)_投组5']]
combined_df.to_csv('组合收益对比.csv')
print(combined_df)

# 计算相关性矩阵（按列之间相关性）
correlation_matrix = combined_df.corr()
print("\n===== 投资组合相关性矩阵 =====")
print(correlation_matrix)
correlation_matrix.to_csv('组合收益相关性.csv')

3.2 细分各项收益来源相关性分析

In [ ]:
# 交易持仓的盈亏来源，按照 资产名称分类
# ticker范围: {'交易持仓','资金费','交易手续费','利息','奖励金'}
ticker = '交易手续费' 
df1 = pd.read_csv(f'p1_{ticker}.csv')[['币种类别','期间总盈亏($)']].iloc[:-1]
df2 = pd.read_csv(f'p2_{ticker}.csv')[['币种类别','期间总盈亏($)']].iloc[:-1]
df3 = pd.read_csv(f'p3_{ticker}.csv')[['币种类别','期间总盈亏($)']].iloc[:-1]
df4 = pd.read_csv(f'p4_{ticker}.csv')[['币种类别','期间总盈亏($)']].iloc[:-1]
df5 = pd.read_csv(f'p5_{ticker}.csv')[['币种类别','期间总盈亏($)']].iloc[:-1]

# 合并5个投资组合
combined_df = df1.merge(df2, on='币种类别', how='outer', suffixes=('_投组1', '_投组2'))
df3 = df3.rename(columns={ '期间总盈亏($)':'期间总盈亏($)_投组3','盈亏占比(%)': '盈亏占比(%)_投组3'})
df4 = df4.rename(columns={ '期间总盈亏($)':'期间总盈亏($)_投组4','盈亏占比(%)': '盈亏占比(%)_投组4'})
df5 = df5.rename(columns={ '期间总盈亏($)':'期间总盈亏($)_投组5','盈亏占比(%)': '盈亏占比(%)_投组5'})

combined_df = combined_df.merge(df3, on='币种类别', how='outer')
combined_df = combined_df.merge(df4, on='币种类别', how='outer')
combined_df = combined_df.merge(df5, on='币种类别', how='outer')


combined_df = combined_df.fillna(0)
combined_df.set_index('币种类别', inplace=True)
combined_df = combined_df[['期间总盈亏($)_投组1','期间总盈亏($)_投组2','期间总盈亏($)_投组3','期间总盈亏($)_投组4','期间总盈亏($)_投组5']]
combined_df.to_csv(f'组合{ticker}收益对比.csv')
print(combined_df)

# 计算相关性矩阵（按列之间相关性）
correlation_matrix = combined_df.corr()
print("\n===== 投资组合相关性矩阵 =====")
print(correlation_matrix)
correlation_matrix.to_csv(f'组合{ticker}收益相关性.csv')